In [2]:
# install important libraries
import os
os.add_dll_directory('C:\Program Files\IBM\clidriver\\bin')
import pandas as pd
import numpy as np
import ibm_db
from typing import List

In [3]:
from configparser import ConfigParser

In [4]:
parser = ConfigParser()
_ = parser.read('config.cfg')

In [11]:
# configurations of ibm db2
host = parser.get('my_creds','host')
dbname=parser.get('my_creds','dbname')
username =parser.get('my_creds','user')
password =parser.get('my_creds','password')
port = parser.get('my_creds','port')

DIR_NAME = 'Datasets'

In [6]:
# Get All Files From The Current Directory
def get_csv_files()->List:
    return [file for file in os.listdir(os.getcwd()) if file.endswith('.csv')]

In [7]:
# Move All Files in a directory
def move_files_in_directory(csv_files: List, dirname:str=DIR_NAME)->None:
    try:
        os.system(f"mkdir {dirname}")
    except:
        print("Error While Creating Dir...")
    
    for csv in csv_files:
        os.system(f"mv '{csv}' {dirname}")
    return

In [8]:
# Read All Files and Create DF
def create_df(csv_files: List, dirname:str):
    path = os.getcwd()+'/'+dirname+'/'
    df= {}
    for file in csv_files:
        try:
            df[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1") #if utf-8 encoding error
    return df

In [9]:
# Clean Table Name
def clean_table_name(filename: str)->str:
    
    #rename csv, force lower case, no spaces, no dashes
    clean_tbl_name = filename.lower().replace(" ", "").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
    return f"{clean_tbl_name.split('.')[0]}"

In [24]:
# Clean Columns Name
def clean_colname(dataframe):
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace(".","_").replace("$","").replace("%","") for x in dataframe.columns]
    
    #processing data
    replacements = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    return col_str, dataframe.columns

In [9]:
# Upload Data On Cloud
def upload_to_db(host, dbname, user, password, tbl_name, col_str, file, dataframe, dataframe_columns):
    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')
    
    #drop table with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created successfully'.format(tbl_name)) 
    
    #insert values to table

    #save df to csv
    dataframe.to_csv(file, header=dataframe_columns, index=False, encoding='utf-8')

    #open the csv file, save it as an object
    my_file = open(file)
    print('file opened in memory')
    
    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')
    
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    cursor.close()
    print('table {0} imported to db completed'.format(tbl_name))

    return

In [10]:
# Calling Functionality
#configure environment and create main df
csv_files = get_csv_files()
move_files_in_directory(csv_files, dataset_dir)
df = create_df(dataset_dir, csv_files)

for file in csv_files:

    #call dataframe
    dataframe = df[file]

    #clean table name
    tbl_name = clean_tbl_name(file)
    
    #clean column names
    col_str, dataframe.columns = clean_colname(dataframe)
    
    #upload data to db   
    upload_to_db(host, 
                 dbname, 
                 user, 
                 password, 
                 tbl_name, 
                 col_str, 
                 file=file, 
                 dataframe=dataframe, 
                 dataframe_columns=dataframe.columns)

NameError: name 'csv_files' is not defined

In [10]:
conn_str=f'database={dbname};hostname={host};port={port};protocol=tcpip;uid={username};pwd={password}'

In [12]:
ibm_db_conn = ibm_db.connect(conn_str,'','')

Exception: [IBM][CLI Driver] SQL1042C  An unexpected system error occurred.  SQLSTATE=58004 SQLCODE=-1042